In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from load_logs import *
from evaluation import *
from features import *
from visualize import *

In [ ]:
logs, log_labels = read_logs_and_labels("./Saved/logs.txt", "./Saved/labels.txt")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

## Grid Search

In [ ]:
metrics = ["euclidean", "cosine"] #"brute" and "auto" were presenting the exact same results.
#leaf_size = [10, 20, 30, 40, 50] #all leaf sizes were prducing the same results
eps = [0.1, 0.2, 0.3, 0.4, 0.5]
min_samples = [100, 200, 300, 400, 500]

In [ ]:
def grid_search(logs, labels_, gram, min_df):
     
    X = get_features(logs, gram, min_df)
    X = X.toarray()
    
    idxs = np.where(np.all(X == 0, axis=1))
    
    X = np.delete(X, idxs, axis=0)
    labels_ = np.delete(labels_, idxs)
    
    result_header = ["Metrics", "Eps", "Min Samples", "VMeasure", "Fowlkes-Mallows", "Clusters (No)"]
    results = []
    
    labels = []
    ls = 0
    
    for metric in metrics:
        for ep in eps:
            for min_s in min_samples:
                model = DBSCAN(eps=ep,min_samples=min_s,algorithm="auto",metric=metric)
                model.fit(X)

                labels = model.labels_

            #plt.figure()
            #plt.suptitle("DBSCAN(gram=%d,eps=%d,min_samples=%d,leaf_size=%d,algorithm=%s)" % (gram,ep,min_s,ls,alg), size=14)

                vm = evaluate_vmeasure(labels_, labels)
                fm = evaluate_fm(labels_, labels)

                lst1 = set(labels)

                result = [metric, ep, min_s, vm, fm, len(lst1)-1]
                results.append(result)
                tab_results(result_header, results) 

                
    tab_results(result_header, results)         
            

In [ ]:
grid_search(logs, log_labels, 2, 70)

In [ ]:
grid_search(logs, log_labels, 2, 100)

In [ ]:
grid_search(logs, log_labels, 3, 90)

In [ ]:
grid_search(logs, log_labels, 3, 100)

## Bigram Feature Vectorizer

In [ ]:
eval_results = []

In [ ]:
labels_ = log_labels

X = get_features(logs, 2, 70)
X = X.toarray()

idxs = np.where(np.all(X == 0, axis=1))

X = np.delete(X, idxs, axis=0)
labels_ = np.delete(labels_, idxs)

In [ ]:
model = DBSCAN(eps=0.4,min_samples=200,algorithm="auto",metric="cosine")
model.fit(X)

In [ ]:
labels = model.labels_

In [ ]:
lst1 = set(labels)
lst1

In [ ]:
plot_clusters("DBSCAN Bigram Clustering using UMAP", X, labels)

In [ ]:
results = evaluate_clustering('DBSCAN Bigram Clustering ', X, labels_, labels)
print(results)

In [ ]:
eval_results.append(results)

In [ ]:
np.count_nonzero(labels == -1)

## Trigram Feature Vectorizer

In [ ]:
labels_ = log_labels

X = get_features(logs, 3, 90)
X = X.toarray()

idxs = np.where(np.all(X == 0, axis=1))

X = np.delete(X, idxs, axis=0)
labels_ = np.delete(labels_, idxs)

In [ ]:
model = DBSCAN(eps=0.5,min_samples=100,algorithm="auto",metric="cosine")
model.fit(X)

In [ ]:
labels = model.labels_

In [ ]:
lst1 = set(labels)
lst1

In [ ]:
plot_clusters("DBSCAN Trigram Clustering using UMAP (100)", X, labels)

In [ ]:
results = evaluate_clustering('DBSCAN Trigram Clustering (100)', X, labels_, labels)
print(results)

In [ ]:
eval_results.append(results)

In [ ]:
tabulate_results(eval_results)